<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/generation_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 582 kB 30.8 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 2.8 MB 55.3 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 419 kB 60.1 MB/s 
     |████████████████████████████████| 141 kB 22.3 MB/s 
     |████████████████████████████████| 3.3 MB 77.5 MB/s 
     |████████████████████████████████| 880 kB 50.6 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 96 kB 6.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=af7548bc0d46b7506fb6bed37b47d469dac67dfd69a5e9e577d5201ef6e8cdea
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Su

##모델 로딩

In [3]:
from transformers import GPT2LMHeadModel
model=GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [4]:
from transformers import PreTrainedTokenizerFast
tokenizer=PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


##KoGPT2모델에 넣을 입력값인 컨텍스트(프롬프트) 만들기

In [7]:
input_ids=tokenizer.encode("안녕하세요",return_tensors='pt') #return_tensors 인자를 pt로 주면 인덱싱 결과를 파이토치의 텐서자료형으로 반환
input_ids

tensor([[25906,  8702,  7801,  8084]])

##그리디 서치(Greeady Search)
다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴.  
여러 번 수행하여도 생성 결과가 바뀌지 않음.


In [8]:
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False, #확률값이 높은 단어를 다음 단어로 결정
      min_length=10, #생성 최소 길이. 이보다 짧은 구간에서 스폐셜 토큰이 등장하여 생성이 멈추면 해당 토큰이 나올 확률을 0으로 수정하여 적어도 해당 길이를 만족할 때까지는 문장생성이 종료되지 않도록 함
      max_length=50, # 생성 최대 길이. 이보다 길거나 짧더라도 EOS와 같은 스폐셜 토큰이 나타나면 생성 중지
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]])) #토큰 인덱스를 문장으로 복원

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



##빔 서치(Beam Search)

In [9]:
with torch.no_grad():
  generated_ids=model.generate(
      input_ids,
      do_sample=False, #확률값이 높은 단어를 다음 단어로 결정
      min_length=10,
      max_length=50,
      num_beams=3 #빔 크기 3
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


##반복 줄이기(1): 반복되는 n-gram사이즈 지정
위의 예시를 보면 "그럼, 그건 뭐예요?"이 반복되는데, 이를 아래와 같이 지정해 반복을 방지.   
3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제.

In [10]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3, #토큰 3개 이상 반복될 경우 3번째 토큰 확률을 0으로 변경
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


##반복 줄이기(2):리피티션 페널티  
모델의 출력 로짓(logit, 소프트맥스 변환 전 벡터)을 일부 가공해 적용.  
로짓 요솟값 > 0 : 원래 요솟값 / repetition_penalty  
로짓 요솟값 < 0 : 원래 요솟값 * repetition_penalty

In [11]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0, #페널티 값으로 1을 주면, 아무런 페널티도 적용되지 않아 그리디 서치와 같은 효과를 냄
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [12]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [13]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [14]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


##탑-K 샘플링(Top-K Sampling)
top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있으므로 매 시행 때마다 생성 결과가 달라짐. k는 1 이상의 값을 가져야함.

In [16]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True, #샘플링 방식을 사용하여 다음 토큰 생성
        min_length=10,
        max_length=50,
        top_k=50, #k를 50으로 설정
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요!
저희는 오피스텔에서 자주 들르다보니 그런 컷을 많이 볼 수 있는 것 같아요.
얼마 전에 우리 집 앞인데 한 달 전부터 사진 찍기 시작했어요.
일주일에 한 번씩 이렇게 사진을 찍


##템퍼러쳐 스케일링(Temperature Scaling)
모델의 다음 토큰 확률 분포를 대소 관계의 역전 없이 분포의 모양만을 바꿔서 문자을 다양하게 생성하는 기법.  
모델의 출력 로짓(소프트맥스 변환 전 벡터)의 모든 요솟값을 temperature로 나누는 방식으로 적용   
탑-K 샘플링이나 탑-P 샘플링과 함께 적용해야 의미가 있음  
temperature < 1 : 확률 분포 모양이 뾰족해짐 → 원래 컸던 확률은 더 커지고, 작았던 확률은 더 작아짐    
temperature = 1 : 모델 출력 분포를 그대로 사용하지만, 샘플링 방식을 채택했디 때문에 생성할 때마다 다른 문장이 생성   
temperature > 1 : 확률분포가 평평해짐. 즉 원래 컸던 확률과 작았던 확률 사이의 갭이 줄어듦. 다양한 문장이 생성될 가능성이 높아지지만 생성 문장의 품질이 나빠질 수 있음

In [17]:
#Top-K Sampling + Temperature Scaling
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [18]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요'.
'그럴 줄 알았으면'.
'어둠이 몰려왔다.</d> 현대자동차가 미국 고속도로 교통안전보험협회(IIHS)로부터 안전보험 부문 최고 등급인 '톱(TOP)'을 획득했다고


In [19]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_k=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요'를 말하면서도 그 의미를 전혀 모르며 침묵. 게다가 문고문을 했던 정철의 사장이 지금 자기에게 사과하라면서 화는 치른다.
안대근 같은 문건을 어떻게 알게되나 알 수는 없으나 그가 내게 이런 책을


##탑-P 샘플링(Top-P Sampling, Nucleus Sampling)
확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이 p이상인 최소 개수의 토큰 집합 가운데 하나를 다음 토큰으로 선택하는 기법

In [20]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?)" "저도 저도 모르겠고요. 제 말입니다. 저도 그렇고요, 잘 모르겠습니다" 등등등. 물론 우리 사회에는 아직 낯선 것들이 많다.
오래 전에 어떤 분이 이렇게 말씀하셨


In [21]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01, #top_p가 0에 가까울 경우 그리디서치와 비슷해짐
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



##종합 적용
실습에 사용하고 있는 허깅페이스(huggingface) 라이브러리의 구현상 적용 순서

- _get_logits_processor
  - RepetitionPenalty
  - NoRepeatNGramLogits
  - MinLengthLogits
- _get_logits_warper
  - TemperatureLogits
  - TopKLogits
  - TopPLogits



In [23]:
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True, #샘플링 방식
        min_length=10,
        max_length=50,
        repetition_penalty=1.5, #반복 줄이기
        no_repeat_ngram_size=3, #반복 줄이기
        temperature=0.9, #템퍼러쳐 스케일링
        top_k=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요!
오늘 소개해 드리고 싶은데요.
지난 주 금요일 밤 11시 50분에 방송이 나갔는데요?
저는 좀 전에 나왔던 그 뉴스 채널에 이어서 오늘도 다시 생방송으로 준비되어 있더라고요,

